## Greedy Distance Connector

This notebook demonstrates example usage of the greedy distance connector model, which can be used to estimate the length of fiber cable needed to connect schools of interest to the internet.

Let's first load in existing school data.

In [ ]:
from giga.schemas.school import GigaSchoolTable
from giga.utils.globals import *
from giga.data.store.stores import COUNTRY_DATA_STORE as data_store
from giga.data.store.stores import SCHOOLS_DATA_STORE as schools_data_store
import os

country = "RWA" #Rwanda
workspace = "workspace"
country_dir = os.path.join(workspace,country)

schools_file = os.path.join(country_dir,SCHOOLS_FILE)
schools = GigaSchoolTable.from_csv(schools_file)
u_schools = [school for school in schools.schools if not school.connected]
unconn_schools = GigaSchoolTable(schools=u_schools)
coords = unconn_schools.to_coordinates()

In [ ]:
from giga.schemas.geo import UniqueCoordinateTable

fiber_file = os.path.join(country_dir,FIBER_FILE)
fiber_coords = UniqueCoordinateTable.from_csv(fiber_file)

Lets take a look at what these schools look like.

In [ ]:
from giga.viz.notebooks.fiber import plot_fiber_map, plot_coordinate_map, default_rwanda_map

plot_coordinate_map(coords, coordinate_name='School', color='#43adde', coordinate_radius=4, m=default_rwanda_map())

Our objective is to estimatee the cost of connect fiber nodes (green) to schools (blue).
The primary cost driver is laying new fiber, so we want to estimate the total distance of fiber lines that would be needed to conenct the schools in the region of interest.
The previous approach simply used the distance of the nearest fiber node to do this estimate - this creates a significant overestimation of costs.
The new approach leverages economies of scale, by assuming that a newly connected school can serve as another node that other schools can connect to - a likely accurate assumption for most real-world schools.
You can see what this looks like for a simple scenario of schools/fiber nodes for the naive approach we used before and the updated approach.

Note: we are creating a simple toy scenario as an example with 50 schools an 10 fiber nodes using existing school data.

In [ ]:
from giga.viz.notebooks.fiber import plot_fiber_map, default_rwanda_map

# let's create a simple set of example schools and fiber nodes by picking arbitrarily from our existing school dataset.
schools = coords[0:50] # in blue
fiber = fiber_coords.coordinates

plot_fiber_map(fiber, schools, m=default_rwanda_map())

Let's try the connector model by first running it with the `dynamic_connect` option set to `False`.
This will create connection between only the existing fiber network and the schools without treating the new schools as potential fiber nodes that can be used to reduce connectivity costs through economies of scale.

In [ ]:
from giga.schemas.distance_cache import SingleLookupDistanceCache, MultiLookupDistanceCache, GreedyConnectCache

fiber_cache_file = os.path.join(country_dir,FIBER_CACHE_FILE)
schools_cache_file = os.path.join(country_dir,SCHOOLS_CACHE_FILE)
connected_cache = SingleLookupDistanceCache.from_json(fiber_cache_file)
unconnected_cache = MultiLookupDistanceCache.from_json(schools_cache_file)
cache = GreedyConnectCache()
cache.connected_cache = connected_cache
cache.unconnected_cache = unconnected_cache

In [ ]:
from giga.models.nodes.graph.greedy_distance_connector import GreedyDistanceConnector
from giga.viz.notebooks.fiber import plot_fiber_connections, default_rwanda_map

m = GreedyDistanceConnector(fiber, dynamic_connect=False, progress_bar=True, distance_cache=cache)
distances_baseline = m.run(schools)
plot_fiber_connections(fiber, schools, distances_baseline, m=default_rwanda_map())

Let's now see what kind of results the connection model produces when the `dynamic_connect` flag is set to `True`.
This model will connect unconnected schools to both fiber nodes and other schools that have been connected.

In [ ]:
from giga.models.nodes.graph.greedy_distance_connector import GreedyDistanceConnector
from giga.viz.notebooks.fiber import plot_fiber_connections, default_rwanda_map

m = GreedyDistanceConnector(fiber, dynamic_connect=True, progress_bar=True, distance_cache=cache)
distances_economies_of_scale = m.run(schools)
plot_fiber_connections(fiber, schools, distances_economies_of_scale, m=default_rwanda_map())

Let's use an interactive plot below to see how the connections are created as the algorithm behind the model runs.
The model connects 50 schools to the fiber network, so we'll see a map of the connections over 50 iterations (one for each school).

In [ ]:
import geopandas as gpd
from giga.viz.notebooks.fiber import interactive_connection_history

border_file = '../parameter_workspace/TM_WORLD_BORDERS-0.3/TM_WORLD_BORDERS-0.3.shp'
borders = gpd.read_file(border_file)
b = borders[borders["NAME"] == 'Rwanda']



In [ ]:
interactive_connection_history(b, fiber, schools, distances_economies_of_scale)

Let's summarize the results from each of our connection runs in a table.

In [ ]:
import pandas as pd
import numpy as np

db = np.round(sum(list(map(lambda x: x.distance, distances_baseline))) / 1000.0, 2)
ds = np.round(sum(list(map(lambda x: x.distance, distances_economies_of_scale))) / 1000.0, 2)

pd.DataFrame([{'Model Type': 'Baseline', 'Schools': len(schools), 'Fiber Nodes': len(fiber), 'Distance Fiber (km)': db},
              {'Model Type': 'Economies of Scale', 'Schools': len(schools), 'Fiber Nodes': len(fiber), 'Distance Fiber (km)': ds}])